# PCM MODULATION

### Import libraries

In [5]:
import numpy as np
import matplotlib.pyplot as plt

### Input Analog Singnal

In [6]:
sampling_rate = 1000  # Hz
duration = 1.0       # sec
carrier_frequency = 50  # Hz
t = np.linspace(0, duration, int(sampling_rate * duration), endpoint=False)  # Time vector
input_signal = np.sin(2 * np.pi * 5 * t)  # Input signal 5 Hz

### Carrier Singnal (square wave)

In [14]:
def carrier(frequency, sampling_rate, duration):
    t = np.linspace(0, duration, int(sampling_rate * duration), endpoint=False)
    carrier_signal = 0.5 * (1 + np.sign(np.sin(2 * np.pi * frequency * t)))
    return carrier_signal
    
carrier_signal = carrier(carrier_frequency, sampling_rate, duration) # generating carrier signal

### Simple PAM Modulation   VS   Flat-Top PAM Modulation

In [11]:
def Simple_pam(signal, carrier_signal):
    normalized_signal = (signal - np.min(signal)) / (np.max(signal) - np.min(signal))
    pam_signal = normalized_signal * carrier_signal
    return pam_signal
    
pam_signal = Simple_pam(input_signal, carrier_signal)

def flat_top_pam(signal, carrier_frequency, sampling_rate):
    normalized_signal = (signal - np.min(signal)) / (np.max(signal) - np.min(signal))
    flat_top_pam_signal = np.zeros_like(signal)
    period = int(sampling_rate / carrier_frequency)
    for i in range(0, len(signal), period):
        flat_top_pam_signal[i:i + period // 4] = normalized_signal[i]  # Short pulse

    return flat_top_pam_signal

flat_top_pam_signal = flat_top_pam(input_signal, carrier_frequency, sampling_rate)

### PFM Modulation

In [12]:
def pfm(signal, carrier_frequency, sampling_rate):
    normalized_signal = (signal - np.min(signal)) / (np.max(signal) - np.min(signal))
    pfm_signal = np.zeros_like(signal)
    phase = 0
    for i in range(len(signal)):
        frequency = carrier_frequency + 40 * normalized_signal[i]
        phase += 2 * np.pi * frequency / sampling_rate
        pfm_signal[i] = 0.5 * (1 + np.sign(np.sin(phase)))
    return pfm_signal
pfm_signal = pfm(input_signal, carrier_frequency, sampling_rate)

### PPM Modulation

In [15]:
def ppm(signal, carrier_frequency, sampling_rate, pulse_width_samples):
    normalized_signal = (signal - np.min(signal)) / (np.max(signal) - np.min(signal))
    ppm_signal = np.zeros_like(signal)
    period = int(sampling_rate / carrier_frequency)
    
    for i in range(len(signal)):
        if i % period == 0:
            shift = int(normalized_signal[i] * period)
            ppm_signal[i + shift:i + shift + pulse_width_samples] = 1

    return ppm_signal
pulse_width_samples = int(0.010 * sampling_rate) # Adjasting the width of the PPM signal's pulses
ppm_signal = ppm(input_signal, carrier_frequency, sampling_rate, pulse_width_samples)


### Plotting signals

In [ ]:
plt.figure(figsize=(12, 14))

# Input Signal
plt.subplot(5, 1, 1)
plt.plot(t, input_signal, color='blue')  # Blue for input signal
plt.title("Input Signal (5 Hz Sine Wave)")
plt.xlabel("Time [s]")
plt.ylabel("Amplitude")

# Carrier Signal
plt.subplot(5, 1, 2)
plt.plot(t, carrier_signal, color='green')  # Green for carrier signal
plt.title("Carrier Signal (50 Hz Square Wave)")
plt.xlabel("Time [s]")
plt.ylabel("Amplitude")

# Regular PAM Signal
plt.subplot(5, 1, 3)
plt.plot(t, pam_signal, color='red')  # Red for PAM signal
plt.title("Regular PAM Signal (Pulse Amplitude Modulation)")
plt.xlabel("Time [s]")
plt.ylabel("Amplitude")

# Discrete Flat-Top PAM Signal
plt.subplot(5, 1, 4)
plt.step(t, flat_top_pam_signal, where='post', color='orange')  # Orange for flat-top PAM signal
plt.title("Discrete Flat-Top PAM Signal")
plt.xlabel("Time [s]")
plt.ylabel("Amplitude")

# PPM Signal
plt.subplot(5, 1, 5)
plt.plot(t, ppm_signal, color='purple')  # Purple for PPM signal
plt.title("PPM Signal (Pulse Position Modulation)")
plt.xlabel("Time [s]")
plt.ylabel("Amplitude")

plt.tight_layout()
plt.show()
